In [2]:
!pip install boto3

     |████████████████████████████████| 139 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 471 kB/s  eta 0:00:01
     |████████████████████████████████| 13.5 MB 35 kB/s  eta 0:00:01    |██▍                             | 1.0 MB 4.2 MB/s eta 0:00:03     |███████▋                        | 3.2 MB 4.2 MB/s eta 0:00:03     |███████████████████████████████▋| 13.3 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 144 kB 1.5 MB/s eta 0:00:01
ERROR: aiobotocore 2.21.1 has requirement botocore<1.37.2,>=1.37.0, but you'll have botocore 1.37.33 which is incompatible.
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.1
    Uninstalling botocore-1.37.1:
      Successfully uninstalled botocore-1.37.1


In [4]:
import boto3
from botocore.client import Config
import os
from dotenv import load_dotenv

# Configurações do MinIO
MINIO_BUCKET = "ingestion"
DATA_FOLDER = "data"

# Carregar variáveis do arquivo .env
load_dotenv()

s3_endpoint = os.getenv("S3_ENDPOINT")
s3_access_key = os.getenv("S3_ACCESS_KEY")
s3_secret_key = os.getenv("S3_SECRET_KEY")

# Cria cliente do MinIO com boto3
s3 = boto3.client(
    "s3",
    endpoint_url=s3_endpoint,
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
    config=Config(signature_version="s3v4"),
    region_name="us-east-1"  # O MinIO não usa regiões, mas o boto3 exige um valor
)

# Verifica se o bucket existe, se não, cria
buckets = [bucket['Name'] for bucket in s3.list_buckets()['Buckets']]
if MINIO_BUCKET not in buckets:
    s3.create_bucket(Bucket=MINIO_BUCKET)
    print(f"🪣 Bucket '{MINIO_BUCKET}' criado.")
else:
    print(f"🪣 Bucket '{MINIO_BUCKET}' já existe.")

# Envia os arquivos CSV para o bucket
for filename in os.listdir(DATA_FOLDER):
    if filename.endswith(".csv"):
        filepath = os.path.join(DATA_FOLDER, filename)
        print(f"📤 Enviando: {filename}")
        with open(filepath, "rb") as f:
            s3.upload_fileobj(f, MINIO_BUCKET, filename)
        print(f"✅ Enviado: {filename}")

print("🎉 Todos os arquivos CSV foram enviados com sucesso para o MinIO!")


🪣 Bucket 'ingestion' já existe.
📤 Enviando: clientes.csv
✅ Enviado: clientes.csv
📤 Enviando: pedido.csv
✅ Enviado: pedido.csv
📤 Enviando: stgPedidos_01.csv
✅ Enviado: stgPedidos_01.csv
📤 Enviando: stgPedidos_02.csv
✅ Enviado: stgPedidos_02.csv
📤 Enviando: stgPedidos_03.csv
✅ Enviado: stgPedidos_03.csv
📤 Enviando: stgPedidos_04.csv
✅ Enviado: stgPedidos_04.csv
📤 Enviando: stgPedidos_05.csv
✅ Enviado: stgPedidos_05.csv
📤 Enviando: stgPedidos_06.csv
✅ Enviado: stgPedidos_06.csv
📤 Enviando: stgPedidos_07.csv
✅ Enviado: stgPedidos_07.csv
📤 Enviando: stgPedidos_08.csv
✅ Enviado: stgPedidos_08.csv
📤 Enviando: stgPedidos_09.csv
✅ Enviado: stgPedidos_09.csv
📤 Enviando: stgPedidos_10.csv
✅ Enviado: stgPedidos_10.csv
🎉 Todos os arquivos CSV foram enviados com sucesso para o MinIO!


In [1]:
import boto3
from botocore.client import Config
import os
import logging
import json
from dotenv import load_dotenv

# -------------------------
# Configuração do Logging
# -------------------------
def setup_logger():
    logger = logging.getLogger("minio_upload")
    logger.setLevel(logging.INFO)

    handler = logging.StreamHandler()
    formatter = logging.Formatter(
        fmt='{"level": "%(levelname)s", "message": "%(message)s"}'
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)

    return logger

logger = setup_logger()

# -------------------------
# Configurações
# -------------------------
MINIO_BUCKET = "ingestion"
#DATA_FOLDER = "data"
DATA_FOLDER = "/opt/notebook/Engineering/data"

# Carregar variáveis do arquivo .env
load_dotenv()

s3_endpoint = os.getenv("S3_ENDPOINT")
s3_access_key = os.getenv("S3_ACCESS_KEY")
s3_secret_key = os.getenv("S3_SECRET_KEY")

# Cria cliente do MinIO com boto3
s3 = boto3.client(
    "s3",
    endpoint_url=s3_endpoint,
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
    config=Config(signature_version="s3v4"),
    region_name="us-east-1"
)

# Verifica se o bucket existe
buckets = [bucket['Name'] for bucket in s3.list_buckets()['Buckets']]
if MINIO_BUCKET not in buckets:
    s3.create_bucket(Bucket=MINIO_BUCKET)
    logger.info(f"Bucket '{MINIO_BUCKET}' criado.")
else:
    logger.info(f"Bucket '{MINIO_BUCKET}' já existe.")

# Lista arquivos .csv
csv_files = [f for f in os.listdir(DATA_FOLDER) if f.endswith(".csv")]

# Se não houver arquivos, encerra
if not csv_files:
    logger.info("Nenhum arquivo .csv encontrado. Processo encerrado.")
    exit(0)

# Envia os arquivos e remove após envio
for filename in csv_files:
    filepath = os.path.join(DATA_FOLDER, filename)
    logger.info(f"Iniciando upload do arquivo: {filename}")

    try:
        with open(filepath, "rb") as f:
            s3.upload_fileobj(f, MINIO_BUCKET, filename)
        logger.info(f"Arquivo enviado com sucesso: {filename}")

        os.remove(filepath)
        logger.info(f"Arquivo removido após upload: {filename}")
    except Exception as e:
        logger.error(f"Erro ao processar {filename}: {str(e)}")

logger.info("Todos os arquivos CSV foram processados com sucesso.")


{"level": "INFO", "message": "Bucket 'ingestion' já existe."}
{"level": "INFO", "message": "Iniciando upload do arquivo: clientes_01.csv"}
{"level": "INFO", "message": "Arquivo enviado com sucesso: clientes_01.csv"}
{"level": "INFO", "message": "Arquivo removido após upload: clientes_01.csv"}
{"level": "INFO", "message": "Iniciando upload do arquivo: clientes_02.csv"}
{"level": "INFO", "message": "Arquivo enviado com sucesso: clientes_02.csv"}
{"level": "INFO", "message": "Arquivo removido após upload: clientes_02.csv"}
{"level": "INFO", "message": "Iniciando upload do arquivo: clientes_03.csv"}
{"level": "INFO", "message": "Arquivo enviado com sucesso: clientes_03.csv"}
{"level": "INFO", "message": "Arquivo removido após upload: clientes_03.csv"}
{"level": "INFO", "message": "Iniciando upload do arquivo: clientes_04.csv"}
{"level": "INFO", "message": "Arquivo enviado com sucesso: clientes_04.csv"}
{"level": "INFO", "message": "Arquivo removido após upload: clientes_04.csv"}
{"level": 